# Lesson 1.3: Core Concepts in LangChain

---

This lesson will introduce the core components of LangChain, the fundamental "building blocks" you'll use to create powerful LLM applications. Understanding each of these concepts is key to grasping how LangChain works and how to combine them.

## 1. Models

In LangChain, "Models" are the interfaces for interacting with various AI models, especially Large Language Models (LLMs) and embedding models. LangChain provides a unified interface so you can easily switch between different model providers.

### 1.1. LLMs (Large Language Models)

* **Concept:** This is the most basic interface for interacting with large language models. They take a text string as input and return a text string as output. LLMs are typically used for text generation, summarization, translation, etc., where both input and output are plain text.
* **Relationship:** LLMs are the foundation for many other components in LangChain, such as Chains and Agents.
* **Examples:** `OpenAI`, `GooglePalm`, `HuggingFaceHub`.

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import OpenAI

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Initialize the LLM model
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.7)

# Invoke the model
text = "Write a short slogan for an AI startup company."
response = llm.invoke(text)
print(response)

### 1.2. Chat Models

* **Concept:** Chat Models are a type of LLM specifically optimized for conversational interactions. Instead of taking a single text string, they take a list of "messages," each with a role (e.g., `system`, `human`, `ai`), and return a message. This helps the model maintain conversation context better.
* **Relationship:** Often used in chatbot applications where conversation history needs to be maintained. They work well with LangChain's Memory components.
* **Examples:** `ChatOpenAI`, `ChatGoogleGenerativeAI`.

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Initialize Chat Model
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Invoke the model with a list of messages
messages = [
    SystemMessage(content="You are a helpful and friendly AI assistant."),
    HumanMessage(content="Hello, how can I help you?"),
    HumanMessage(content="Tell me a short story about a flying cat.")
]
response = chat_model.invoke(messages)
print(response.content)

### 1.3. Embeddings

* **Concept:** Embeddings are numerical representations (vectors) of text. Each vector captures the semantic meaning of that text. Texts with similar meanings will have their embedding vectors close to each other in a multi-dimensional space. Embeddings are crucial for semantic search, recommendation, and clustering tasks.
* **Relationship:** A core component in information retrieval systems, especially in RAG (Retrieval-Augmented Generation) and Vector Stores.
* **Examples:** `OpenAIEmbeddings`, `HuggingFaceEmbeddings`, `GoogleGenerativeAIEmbeddings`.

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import OpenAIEmbeddings

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Initialize Embeddings model
embeddings_model = OpenAIEmbeddings()

# Create embeddings for text snippets
text1 = "The dog is running in the field."
text2 = "A canine is jogging outdoors."
text3 = "The sky today is very clear."

embedding1 = embeddings_model.embed_query(text1)
embedding2 = embeddings_model.embed_query(text2)
embedding3 = embeddings_model.embed_query(text3)

print(f"Embedding of '{text1}' has length: {len(embedding1)}")
# To check similarity, you can calculate cosine similarity between vectors
# (Not directly in LangChain Embeddings, requires a math library like numpy)


---

## 2. Prompts

Prompts are the input instructions you provide to an LLM to guide its behavior. LangChain offers tools to build and manage Prompts effectively.

### 2.1. Prompt Templates

* **Concept:** Prompt Templates are objects that help you construct prompts in a structured and flexible way. They allow you to define a prompt string with "placeholders" (variables) that you can fill in later. This aids in prompt reusability and easy dynamic content changes.
* **Relationship:** The first component in most Chains, taking input from the user or previous steps and formatting it for the LLM.
* **Examples:** `PromptTemplate`, `ChatPromptTemplate`.

In [ ]:
# Install the library if not already installed
# pip install langchain

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

# Example of a basic PromptTemplate
string_prompt = PromptTemplate.from_template("Tell a short story about {character} in {setting}.")
print(string_prompt.format(character="a dragon", setting="an ancient castle"))

# Example of a ChatPromptTemplate (for Chat Models)
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a creative assistant, specialized in writing fairy tales."),
    HumanMessagePromptTemplate.from_template("Write a story about {character} and {pet}."),
])
print(chat_prompt.format_messages(character="Cinderella", pet="a smart mouse"))

### 2.2. Output Parsers

* **Concept:** Output Parsers help extract and format the LLM's string output into desired data structures (e.g., JSON, lists, Python objects). LLMs often return free-form text, but in many applications, we need structured data for further processing.
* **Relationship:** Often the last component in a Chain, transforming the LLM's output into a usable format for subsequent steps or for the user.
* **Examples:** `StrOutputParser`, `CommaSeparatedListOutputParser`, `PydanticOutputParser`.

In [ ]:
# Install the library if not already installed
# pip install langchain langchain-openai pydantic

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Example 1: StrOutputParser (default if no other parser is specified)
prompt_str = ChatPromptTemplate.from_template("Say 'Hello World!'")
chain_str = prompt_str | chat_model | StrOutputParser()
print(f"StrOutputParser: {chain_str.invoke({})}")

# Example 2: CommaSeparatedListOutputParser
prompt_list = ChatPromptTemplate.from_template("List 3 common fruits, separated by commas.")
chain_list = prompt_list | chat_model | CommaSeparatedListOutputParser()
print(f"CommaSeparatedListOutputParser: {chain_list.invoke({})})")

# Example 3: PydanticOutputParser
class BookInfo(BaseModel):
    title: str = Field(description="The title of the book")
    author: str = Field(description="The author's name of the book")
    publication_year: int = Field(description="The publication year of the book")

parser = PydanticOutputParser(pydantic_object=BookInfo)

prompt_pydantic = ChatPromptTemplate.from_messages([
    ("system", "You are an information extraction assistant. Extract the following book information based on the format:\n{format_instructions}"),
    ("human", "Extract book information from the following text: 'The book 'How to Win Friends and Influence People' by Dale Carnegie, published in 1936, is a classic work on communication skills.'")
]).partial(format_instructions=parser.get_format_instructions())

chain_pydantic = prompt_pydantic | chat_model | parser
book_info = chain_pydantic.invoke({})
print(f"PydanticOutputParser: {book_info.title}, {book_info.author}, {book_info.publication_year}")


---

## 3. Chains

* **Concept:** Chains are sequences of components linked together to perform a specific processing flow. Each component in the chain (e.g., Prompt Template, LLM, Output Parser) processes the output of the previous component as its input. Chains help you build complex workflows in an organized and manageable way.
* **Relationship:** Chains are the primary way to combine Models, Prompts, Parsers, and other components. They are the backbone of most LangChain applications.
* **Examples:** `LLMChain`, `SequentialChain`, and chains built using LCEL (LangChain Expression Language).

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Define Prompt Template
prompt = ChatPromptTemplate.from_template("Summarize the following paragraph into one sentence: {text}")

# Build the Chain using LCEL (LangChain Expression Language)
# This is the modern and flexible way to create chains in LangChain
summarize_chain = prompt | chat_model | StrOutputParser()

# Execute the Chain
long_text = "LangChain is an open-source framework designed to help developers build applications powered by Large Language Models (LLMs) more easily and efficiently. It provides a set of tools, components, and abstractions to simplify complex processes related to LLMs, from prompt management to connecting LLMs with external data sources and tools."
summary = summarize_chain.invoke({"text": long_text})
print(f"Summary: {summary}")


---

## 4. Agents

* **Concept:** Agents are LLMs capable of making decisions and performing actions. Unlike Chains which have a fixed processing flow, Agents use an LLM as their "brain" to reason about what action to take next, which tool to use, and when to stop. They can iterate this process until a goal is achieved.
* **Relationship:** Agents extend the capabilities of LLMs by allowing them to interact with the external world through Tools. They often use Memory to maintain context in complex conversations.
* **Example:** An Agent might search for information on Google, then use a calculator to perform a calculation, and finally answer the user's question.

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai wikipedia
# Note: To run this example, you need to install SerpAPI or use another search tool
# pip install google-search-results # If using SerpAPI

import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY" # If using SerpAPI

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Define Tools that the Agent can use
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [wikipedia] # Add other tools if needed, e.g., SearchTool (SerpAPI)

# Load prompt for ReAct Agent from LangChain Hub
prompt = hub.pull("hwchase17/react")

# Create Agent
agent = create_react_agent(llm, tools, prompt)

# Create Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Execute Agent
print("Agent is processing the question...")
response = agent_executor.invoke({"input": "Who is the current president of the United States? What year was he born?"})
print(f"Agent's answer: {response['output']}")


---

## 5. Memory

* **Concept:** Memory is the component that helps maintain the state and context of a conversation across multiple turns. LLMs are inherently "stateless," meaning they don't remember previous interactions. Memory solves this by storing conversation history and feeding it back to the LLM in subsequent calls.
* **Relationship:** Memory is crucial for chatbot applications and conversational tasks where context is key. It is often integrated into Chains and Agents.
* **Examples:** `ConversationBufferMemory`, `ConversationBufferWindowMemory`, `ConversationSummaryMemory`.

In [ ]:
# Install the library if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Initialize Chat Model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Initialize Memory
memory = ConversationBufferMemory()

# Create Conversation Chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True # To see the conversation history passed to the LLM
)

# Conduct the conversation
print("Starting conversation:")
response1 = conversation.predict(input="Hello, what is your name?")
print(f"AI: {response1}")

response2 = conversation.predict(input="My name is An. Can you tell me about LangChain?")
print(f"AI: {response2}")

response3 = conversation.predict(input="So what kind of applications can it help me build?")
print(f"AI: {response3}")

# View conversation history in Memory
print("\nConversation history in Memory:")
print(memory.buffer)


---

## Lesson Summary

This lesson introduced the core concepts in LangChain, which are the foundation for building complex LLM applications. We learned about **Models** (LLMs, Chat Models, Embeddings) - interfaces for interacting with AI models. **Prompts** (Prompt Templates, Output Parsers) help guide and format LLM input/output. **Chains** are how we connect components to create processing flows. **Agents** enable LLMs to make decisions and perform actions through **Tools**. Finally, **Memory** helps maintain context in long conversations. Mastering these concepts will make it easier for you to design and deploy LangChain-based solutions.